In [ ]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2 MB 1.5 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
jax 0.3.4 requires scipy>=1.2.1, but you have scipy 1.1.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from datetime import datetime
import numpy as np
import cv2
import os
from math import ceil
from scipy.sparse import spdiags
from scipy.misc import imresize
from scipy.optimize import fminbound
from scipy.stats import entropy
from scipy.sparse.linalg import spsolve

In [ ]:
# coding:utf-8

filepath = '/content/drive/MyDrive/Colab Notebooks/image/data/Dog/'
filepath_out = '/content/drive/MyDrive/Colab Notebooks/image/data/processed_dog_2/'

filepath2 = '/content/drive/MyDrive/Colab Notebooks/image/data/Cat/'
filepath_out2 = '/content/drive/MyDrive/Colab Notebooks/image/data/processed_cat_2/'


def computeTextureWeights(fin, sigma, sharpness):
    # print(fin)
    # fin = fin / 255.0

    dt0_v = np.diff(fin, 1, 0)  
    dt0_v = np.concatenate((dt0_v, fin[:1, :] - fin[-1:, :]), axis=0) 

    dt0_h = np.diff(fin, 1, 1)  
    dt0_h = np.concatenate((dt0_h, fin[:, :1] - fin[:, -1:]), axis=1) 

    gauker_h = cv2.filter2D(dt0_h, -1, np.ones((1, sigma)), borderType=cv2.BORDER_CONSTANT)
    gauker_v = cv2.filter2D(dt0_v, -1, np.ones((sigma, 1)), borderType=cv2.BORDER_CONSTANT)

    W_h = 1.0 / (abs(gauker_h) * abs(dt0_h) + sharpness)
    W_v = 1.0 / (abs(gauker_v) * abs(dt0_v) + sharpness)

    return W_h, W_v


def convertCol(tmp):  
    return np.reshape(tmp.T, (tmp.shape[0] * tmp.shape[1], 1))


def solveLinearEquation(IN, wx, wy, lambd):
    #print('IN', IN.shape)
    r, c, ch = IN.shape[0], IN.shape[1], 1
    k = r * c
    dx = -lambd * convertCol(wx) 
    dy = -lambd * convertCol(wy)
    tempx = np.concatenate((wx[:, -1:], wx[:, 0:-1]), 1)  
    tempy = np.concatenate((wy[-1:, :], wy[0:-1, :]), 0)  
    dxa = -lambd * convertCol(tempx)
    dya = -lambd * convertCol(tempy)
    tempx = np.concatenate((wx[:, -1:], np.zeros((r, c - 1))), 1)  
    tempy = np.concatenate((wy[-1:, :], np.zeros((r - 1, c))), 0) 
    dxd1 = -lambd * convertCol(tempx)
    dyd1 = -lambd * convertCol(tempy)
    wx[:, -1:] = 0  
    wy[-1:, :] = 0  
    dxd2 = -lambd * convertCol(wx)
    dyd2 = -lambd * convertCol(wy)

    Ax = spdiags(np.concatenate((dxd1, dxd2), 1).T, np.array([-k + r, -r]), k, k)
    Ay = spdiags(np.concatenate((dyd1, dyd2), 1).T, np.array([-r + 1, -1]), k, k)

    D = 1 - (dx + dy + dxa + dya)
    A = (Ax + Ay) + (Ax + Ay).T + spdiags(D.T, np.array([0]), k, k)

    A = A / 1000.0  

    matCol = convertCol(IN)
    OUT = spsolve(A, matCol, permc_spec="MMD_AT_PLUS_A")
    OUT = OUT / 1000
    OUT = np.reshape(OUT, (c, r)).T
    return OUT


def tsmooth(I, lambd=0.5, sigma=5, sharpness=0.001):
    # print(I.shape)
    wx, wy = computeTextureWeights(I, sigma, sharpness)
    S = solveLinearEquation(I, wx, wy, lambd)
    return S


def rgb2gm(I):
    print('I', I.shape)
    # I = np.maximum(I, 0.0)
    if I.shape[2] and I.shape[2] == 3:
        I = np.power(np.multiply(np.multiply(I[:, :, 0], I[:, :, 1]), I[:, :, 2]), (1.0 / 3))
    return I


def YisBad(Y, isBad):  
    return Y[isBad >= 1]
    # Z = []
    # [rows, cols] = Y.shape
    # for i in range(rows):
    #     for j in range(cols):
    #         if isBad[i, j] >= 122:
    #             Z.append(Y[i, j])
    # return np.array([Z]).T


def applyK(I, k, a=-0.3293, b=1.1258):
    # print(type(I))
    if not type(I) == 'numpy.ndarray':
        I = np.array(I)
    # print(type(I))
    beta = np.exp((1 - (k ** a)) * b)
    gamma = (k ** a)
    BTF = np.power(I, gamma) * beta
    return BTF


def maxEntropyEnhance(I, isBad, mink=1, maxk=10):
    # Y = rgb2gm(np.real(np.maximum(imresize(I, (50, 50), interp='bicubic') / 255.0, 0)))
    Y = imresize(I, (50, 50), interp='bicubic') / 255.0
    Y = rgb2gm(Y)
    # Y = rgb2gm(np.real(np.maximum(cv2.resize(I, (50, 50), interpolation=cv2.INTER_LANCZOS4  ), 0)))
    # import matplotlib.pyplot as plt
    # plt.imshow(Y, cmap='gray');plt.show()

    #print('isBad', isBad.shape)
    isBad = imresize(isBad.astype(int), (50, 50), interp='nearest')
    #print('isBad', isBad.shape)

    # plt.imshow(isBad, cmap='gray');plt.show()

    Y = Y[isBad >= 1]

    # Y = sorted(Y)

    #print('-entropy(Y)', -entropy(Y))

    def f(k):
        return -entropy(applyK(Y, k))

    # opt_k = mink
    # k = mink
    # minF = f(k)
    # while k<= maxk:
    #     k+=0.0001
    #     if f(k)<minF:
    #         minF = f(k)
    #         opt_k = k
    opt_k = fminbound(f, mink, maxk)

    J = applyK(I, opt_k) - 0.01
    return J


def HDR2dark(I, t_our, W):  
    W = 1 - W
    I3 = I * W
    isBad = t_our > 0.8
    J3 = maxEntropyEnhance(I, isBad, 0.1, 0.5) 
    J3 = J3 * (1 - W)  
    fused = I3 + J3  
    return I


def oneHDR(I, mu=0.5, a=-0.3293, b=1.1258):

    I = I / 255.0
    t_b = I[:, :, 0]  
    for i in range(I.shape[2] - 1):  
        t_b = np.maximum(t_b, I[:, :, i + 1])

    t_b2 = imresize(t_b, (256, 256), interp='bicubic', mode='F')  # / 255
    t_our = tsmooth(t_b2)  
    #print('t_our前', t_our.shape)
    t_our = imresize(t_our, t_b.shape, interp='bicubic', mode='F')  # / 255
    #print('t_our后', t_our.shape)

    t = np.ndarray(I.shape)
    for ii in range(I.shape[2]):
        t[:, :, ii] = t_our
    #print('t', t.shape)

    W = t ** mu  

    #print('W', W.shape)

    I2 = I * W 

    isBad = t_our < 0.5  
    J = maxEntropyEnhance(I, isBad)  
    J2 = J * (1 - W)  
    fused = I2 + J2  

    return fused
    # return res


def test():
    for filename in os.listdir(filepath):
      if 'DS' not in filename:
        inputImg = cv2.imread(filepath + filename)
        outputImg = oneHDR(inputImg)
      # outputImg = outputImg * 255.0
        cv2.imwrite(filepath_out + filename, outputImg * 255)
        print("HDR完成，已保存到本地")

        print('程序结束', str(datetime.now()))

    for filename in os.listdir(filepath2):
      if 'DS' not in filename:
        inputImg = cv2.imread(filepath2 + filename)
        outputImg = oneHDR(inputImg)
      # outputImg = outputImg * 255.0
        cv2.imwrite(filepath_out2 + filename, outputImg * 255)
        print("HDR saved")

        print('done', str(datetime.now()))

    #cv2.imshow('inputImg', inputImg)
    #cv2.imshow('outputImg', outputImg)
    # print(inputImg.dtype,outputImg.dtype)
    # outputImg = outputImg.astype(int)
    # print(inputImg.dtype, outputImg.dtype)
    # compare = np.concatenate((inputImg,outputImg),axis=1)
    # cv2.imshow('compare', compare)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == '__main__':
    print('begin', str(datetime.now()))
    test()

程序开始 2022-04-19 18:08:16.462038


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:191: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:194: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:126: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:135: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:08:18.374737
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:08:22.165711
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:08:25.995936
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:08:28.053221
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:08:33.295045
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:08:41.008347
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:08:51.832980
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:08:53.849233
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:02.391787
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:04.105451
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:06.226098
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:07.618129
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:08.973389
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:10.375091
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:11.759033
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:13.078896
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:14.5093

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:205: RuntimeWarning: invalid value encountered in sqrt


I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:32.247418
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:33.426098
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:34.604159
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:35.837884
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:37.208408
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:38.485139
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:39.766260
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:41.319184
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:42.912518
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:44.166530
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:45.762489
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:47.215789
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:48.817405
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:50.154939
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:51.568804
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:52.740593
I (50, 50, 3)
HDR完成，已保存到本地
程序结束 2022-04-19 18:09:53.9755